# 3. Merging data

Anda akan mempelajari semua tentang `merging` pandas DataFrames. Anda akan menjelajahi berbagai teknik untuk `merging`, dan mempelajari tentang `left joins`, `right joins`, `inner joins`, dan `outer joins`, serta kapan harus menggunakan yang mana. Anda juga akan belajar tentang *ordered merging*, yang berguna ketika Anda ingin *merge* DataFrames dengan kolom yang memiliki urutan default, seperti kolom tanggal-waktu.

## Merging DataFrames

### Merging company DataFrames

Misalkan perusahaan Anda beroperasi di beberapa kota berbeda di bawah beberapa manajer yang berbeda. `revenue` dan `managers` DataFrames berisi informasi parsial yang terkait dengan perusahaan. Yaitu, baris kolom `city` tidak cukup cocok dengan `revenue` dan `managers` (cabang Mendocino belum memiliki pendapatan sejak dibuka dan manajer cabang Springfield baru-baru ini meninggalkan perusahaan).

Jika Anda menjalankan perintah `combined = pd.merge(revenue, managers, on='city')`, berapa banyak baris yang `combined` punya?

In [2]:
# Import pandas
import pandas as pd

# Create DataFrames
revenue = pd.DataFrame({ 'city' : ['Austin', 'Denver', 'Springfield'], 'revenue' : [100, 83, 4] })
managers = pd.DataFrame({ 'city' : ['Austin', 'Denver', 'Mendocino'], 'manager' : ['Charlers', 'Joel', 'Brett'] })

In [4]:
combined = pd.merge(revenue, managers, on='city')
print(combined)
print(combined.shape)

     city  revenue   manager
0  Austin      100  Charlers
1  Denver       83      Joel
(2, 3)


**Note** : Karena pengaturan default untuk `pd.merge()` adalah *inner join`, `combined` akan memiliki 2 baris.

### Merging on a specific column

Latihan ini mengikuti DataFrames terakhir yaitu `revenue` dan `managers` untuk perusahaan Anda. Anda mengharapkan perusahaan Anda tumbuh dan, pada akhirnya, beroperasi di kota-kota dengan nama yang sama di negara bagian yang berbeda. Karena itu, Anda memutuskan bahwa setiap cabang harus memiliki pengenal cabang numerik. Dengan demikian, Anda menambahkan kolom `branch_id` ke kedua DataFrames. Selain itu, kota-kota baru telah ditambahkan ke DataFrames `revenue` dan `managers` juga.

In [5]:
# Create New DataFrames
revenue = pd.DataFrame({ 'city' : ['Austin', 'Denver', 'Springfield', 'Mendocino'], 'branch_id' : [10, 20, 30, 47], 'revenue' : [100, 83, 4, 200] })
managers = pd.DataFrame({ 'city' : ['Austin', 'Denver', 'Mendocino', 'Springfield'], 'branch_id' : [10, 20, 47, 31], 'manager' : ['Charlers', 'Joel', 'Brett', 'Sally'] })

In [6]:
# Merge revenue with managers on 'city': merge_by_city
merge_by_city = pd.merge(revenue, managers, on='city')

# Print merge_by_city
print(merge_by_city)

# Merge revenue with managers on 'branch_id': merge_by_id
merge_by_id = pd.merge(revenue, managers, on='branch_id')

# Print merge_by_id
print(merge_by_id)

          city  branch_id_x  revenue  branch_id_y   manager
0       Austin           10      100           10  Charlers
1       Denver           20       83           20      Joel
2  Springfield           30        4           31     Sally
3    Mendocino           47      200           47     Brett
      city_x  branch_id  revenue     city_y   manager
0     Austin         10      100     Austin  Charlers
1     Denver         20       83     Denver      Joel
2  Mendocino         47      200  Mendocino     Brett


**Note** : Perhatikan bahwa ketika Anda melakukan merge pada `'city'`, DataFrame yang dihasilkan memiliki hasil yang aneh: Di baris 2, kota Springfield memiliki dua ID cabang yang berbeda. Ini karena sebenarnya ada dua kota berbeda bernama Springfield - satu di Negara Bagian Illinois, dan yang lainnya di Missouri. DataFrame `revenue` memiliki satu dari Illinois, dan DataFrame `managers` memiliki satu dari Missouri. Akibatnya, ketika Anda merge pada `'branch_id'`, kedua ini akan dihapus dari DataFrame yang di merge.

### Merging on columns with non-matching labels

Anda terus bekerja dengan DataFrames `revenue` & `managers` dari sebelumnya. Kali ini, seseorang telah mengubah nama bidang `'city'` menjadi `'branch'` di tabel `managers`. Sekarang, ketika Anda mencoba untuk merge DataFrames, sebuah ekspresi dilemparkan:

<pre>
>>> pd.merge(revenue, managers, on='city')
Traceback (most recent call last):
    ... <text deleted> ...
    pd.merge(revenue, managers, on='city')
    ... <text deleted> ...
KeyError: 'city'
</pre>

Dengan ini, akan dibutuhkan sedikit lebih banyak pekerjaan bagi Anda untuk merge atau join nama kota/cabang. Anda harus menentukan parameter `left_on` dan `right_on` dalam panggilan ke `pd.merge()`.

Apakah Anda bisa melakukan merge lebih baik daripada di latihan terakhir? Bagaimana seharusnya baris `Springfield` ditangani?

In [10]:
# Create New DataFrames
revenue = pd.DataFrame({ 'city' : ['Austin', 'Denver', 'Springfield', 'Mendocino'], 'branch_id' : [10, 20, 30, 47], 'state' : ['TX', 'CO', 'IL', 'CA'], 'revenue' : [100, 83, 4, 200] })
managers = pd.DataFrame({ 'branch' : ['Austin', 'Denver', 'Mendocino', 'Springfield'], 'branch_id' : [10, 20, 47, 31], 'state' : ['TX', 'CO', 'CA', 'MO'], 'manager' : ['Charlers', 'Joel', 'Brett', 'Sally'] })

In [12]:
# Merge revenue & managers on 'city' & 'branch': combined
combined = pd.merge(revenue, managers, left_on='city', right_on='branch')

# Print combined
combined

,city,branch_id_x,state_x,revenue,branch,branch_id_y,state_y,manager
0,Austin,10,TX,100,Austin,10,TX,Charlers
1,Denver,20,CO,83,Denver,20,CO,Joel
2,Springfield,30,IL,4,Springfield,31,MO,Sally
3,Mendocino,47,CA,200,Mendocino,47,CA,Brett


**Note** : Penting untuk memperhatikan bagaimana kolom dinamai dalam DataFrames yang berbeda.

### Merging on multiple columns

Strategi lain untuk membuat kota dengan nama yang identik adalah dengan menambahkan informasi tentang negara-negara di mana kota-kota tersebut berada. Untuk tujuan ini, Anda menambahkan kolom `state` ke kedua DataFrames dari latihan sebelumnya. 

Tujuan Anda dalam latihan ini adalah menggunakan `pd.merge()` untuk menggabungkan DataFrames menggunakan beberapa kolom (menggunakan `'branch_id'`, `'city'`, dan `'state'` dalam kasus ini).

Apakah Anda dapat mencocokkan semua cabang perusahaan Anda dengan benar?

In [13]:
# Create DataFrames
revenue = pd.DataFrame({ 'city' : ['Austin', 'Denver', 'Springfield', 'Mendocino'], 'branch_id' : [10, 20, 30, 47], 'revenue' : [100, 83, 4, 200] })
managers = pd.DataFrame({ 'city' : ['Austin', 'Denver', 'Mendocino', 'Springfield'], 'branch_id' : [10, 20, 47, 31], 'manager' : ['Charlers', 'Joel', 'Brett', 'Sally'] })

In [14]:
# Add 'state' column to revenue: revenue['state']
revenue['state'] = ['TX','CO','IL','CA']

# Add 'state' column to managers: managers['state']
managers['state'] = ['TX','CO','CA','MO']

# Merge revenue & managers on 'branch_id', 'city', & 'state': combined
combined = pd.merge(revenue, managers, on=['branch_id', 'city', 'state'])

# Print combined
print(combined)

        city  branch_id  revenue state   manager
0     Austin         10      100    TX  Charlers
1     Denver         20       83    CO      Joel
2  Mendocino         47      200    CA     Brett


**Note** : Anda telah mencocokkan semua cabang dengan benar!

## Joining DataFrames

### Joining by Index

DataFrames `revenue` dan `managers` sekarang akan ditambahkan index menggunakan `'branch_id'`.

Pilih panggilan fungsi di bawah ini yang akan melakukan join DataFrames pada indeks dan mengembalikan 5 baris dengan label indeks `[10, 20, 30, 31, 47]`. Jelajahi masing-masing dari mereka di Shell IPython untuk mendapatkan pemahaman yang lebih baik tentang fungsi tersebut.

In [15]:
# Create New DataFrames
revenue = pd.DataFrame({ 'city' : ['Austin', 'Denver', 'Springfield', 'Mendocino'], 'branch_id' : [10, 20, 30, 47], 'state' : ['TX', 'CO', 'IL', 'CA'], 'revenue' : [100, 83, 4, 200] }).set_index('branch_id')
managers = pd.DataFrame({ 'branch' : ['Austin', 'Denver', 'Mendocino', 'Springfield'], 'branch_id' : [10, 20, 47, 31], 'state' : ['TX', 'CO', 'CA', 'MO'], 'manager' : ['Charlers', 'Joel', 'Brett', 'Sally'] }).set_index('branch_id')

**BENAR**

Panggilan fungsi ini memang mengembalikan 5 baris dengan label indeks `[10, 20, 30, 31, 47]`

In [16]:
# Correct
revenue.join(managers, lsuffix='_rev', rsuffix='_mng', how='outer')

,city,state_rev,revenue,branch,state_mng,manager
branch_id,,,,,,
10,Austin,TX,100.0,Austin,TX,Charlers
20,Denver,CO,83.0,Denver,CO,Joel
30,Springfield,IL,4.0,NaN,NaN,NaN
31,NaN,NaN,NaN,Springfield,MO,Sally
47,Mendocino,CA,200.0,Mendocino,CA,Brett


In [17]:
pd.merge(managers, revenue, how='left')

,branch,state,manager,city,revenue
0,Austin,TX,Charlers,Austin,100.0
1,Denver,CO,Joel,Denver,83.0
2,Mendocino,CA,Brett,Mendocino,200.0
3,Springfield,MO,Sally,NaN,NaN


In [19]:
managers.join(revenue, lsuffix='_mgn', rsuffix='_rev', how='left')

,branch,state_mgn,manager,city,state_rev,revenue
branch_id,,,,,,
10,Austin,TX,Charlers,Austin,TX,100.0
20,Denver,CO,Joel,Denver,CO,83.0
47,Mendocino,CA,Brett,Mendocino,CA,200.0
31,Springfield,MO,Sally,NaN,NaN,NaN


### Choosing a joining strategy

Misalkan Anda memiliki dua DataFrame: 

* `students` - dengan kolom `'StudentID'`, `'LastName'`, `'FirstName'`, dan `'Major'`)
* `midterm_results` - dengan kolom `'StudentID'`, `'Q1'`, `'Q2'`, dan `'Q3'` untuk skor pada pertanyaan jangka menengah).

Anda ingin menggabungkan DataFrames menjadi DataFrame tunggal `grades`, dan dapat dengan mudah menemukan siswa mana yang menulis midterm dan yang tidak (skor pertanyaan tengah semester mereka `'Q1'`, `'Q2'`, & `'Q3'` harus diisi dengan nilai `NaN`).

Anda juga ingin menghapus baris dari `midterm_results` di mana `StudentID` tidak ditemukan pada `students`.

Manakah dari strategi berikut ini yang memberikan hasil yang diinginkan?

**Benar**

* A left join: <pre>grades = pd.merge(students, midterm_results, how='left').</pre>

**Salah**

* A right join: <pre>grades = pd.merge(students, midterm_results, how='right').</pre>
* An inner join: <pre>grades = pd.merge(students, midterm_results, how='inner')</pre>
* An outer join: <pre>grades = pd.merge(students, midterm_results, how='outer')</pre>

### Left & right merging on multiple columns

Anda sekarang memiliki DataFrames tambahan, di samping `revenue` dan `managers` dari latihan sebelumnya, DataFrame `sales` yang merangkum unit yang dijual dari cabang tertentu (diidentifikasi oleh `city` dan `state` tetapi bukan `branch_id`).

Sekali lagi, DataFrame `managers` menggunakan label `branch` pada `city` tidak seperti di dua DataFrames lainnya. Tugas Anda di sini adalah menggunakan *merge left* dan *right* untuk menyimpan data dan mengidentifikasi di mana data hilang.

Dengan menggabungkan `revenue` dan `sales` menggunakan *merge right*, Anda dapat mengidentifikasi nilai `revenue` yang hilang. Di sini, Anda tidak perlu menentukan `left_on` atau `right_on` karena kolom untuk di merge memiliki label yang cocok.

Dengan menggabungkan `sales` dan `managers` menggunakan *merge left*, Anda dapat mengidentifikasi `manager` yang hilang. Di sini, kolom yang akan digabungkan memiliki label yang bertentangan, jadi Anda harus menentukan `left_on` dan `right_on`. Dalam kedua kasus, Anda mencari tahu cara menghubungkan bidang di baris yang mengandung `Springfield`.

In [21]:
# Create New DataFrames
revenue = pd.DataFrame({ 'city' : ['Austin', 'Denver', 'Springfield', 'Mendocino'], 'branch_id' : [10, 20, 30, 47], 'state' : ['TX', 'CO', 'IL', 'CA'], 'revenue' : [100, 83, 4, 200] })
managers = pd.DataFrame({ 'branch' : ['Austin', 'Denver', 'Mendocino', 'Springfield'], 'branch_id' : [10, 20, 47, 31], 'state' : ['TX', 'CO', 'CA', 'MO'], 'manager' : ['Charlers', 'Joel', 'Brett', 'Sally'] })
sales = pd.DataFrame({ 'city' : ['Mendocino', 'Denver', 'Austin', 'Springfield', 'Springfield'], 'state' : ['CA', 'CO', 'TX', 'MO', 'IL'], 'units' : [1, 4, 2, 5, 1] })

# Print data
print(revenue)
print(managers)
print(sales)

          city  branch_id state  revenue
0       Austin         10    TX      100
1       Denver         20    CO       83
2  Springfield         30    IL        4
3    Mendocino         47    CA      200
        branch  branch_id state   manager
0       Austin         10    TX  Charlers
1       Denver         20    CO      Joel
2    Mendocino         47    CA     Brett
3  Springfield         31    MO     Sally
          city state  units
0    Mendocino    CA      1
1       Denver    CO      4
2       Austin    TX      2
3  Springfield    MO      5
4  Springfield    IL      1


In [23]:
# Merge revenue and sales: revenue_and_sales
revenue_and_sales = pd.merge(revenue, sales, how='right', on=['city', 'state'])

# Print revenue_and_sales
print(revenue_and_sales)

# Merge sales and managers: sales_and_managers
sales_and_managers = pd.merge(sales, managers, how='left', left_on=['city', 'state'], right_on=['branch', 'state'])

# Print sales_and_managers
print(sales_and_managers)

          city  branch_id state  revenue  units
0       Austin       10.0    TX    100.0      2
1       Denver       20.0    CO     83.0      4
2  Springfield       30.0    IL      4.0      1
3    Mendocino       47.0    CA    200.0      1
4  Springfield        NaN    MO      NaN      5
          city state  units       branch  branch_id   manager
0    Mendocino    CA      1    Mendocino       47.0     Brett
1       Denver    CO      4       Denver       20.0      Joel
2       Austin    TX      2       Austin       10.0  Charlers
3  Springfield    MO      5  Springfield       31.0     Sally
4  Springfield    IL      1          NaN        NaN       NaN


**Note** : Ini adalah cara yang baik untuk mempertahankan kedua entri `Springfield`.

### Merging DataFrames with outer join

DataFrame yang digunakan `revenue`, `managers`, dan `sales` sama seperti sebelumnya, Dan juga `revenue_and_sales` dan `sales_and_managers`.

DataFrame yang digabung berisi informasi yang cukup untuk membangun DataFrame dengan 5 baris dengan semua informasi yang diketahui disejajarkan dengan benar dan setiap cabang hanya terdaftar sekali. Anda akan mencoba untuk menggabungkan DataFrame yang digabungkan pada semua kunci yang cocok (yang menghitung *inner join* secara default). Anda dapat membandingkan hasilnya dengan *outer join* dan juga *outer join* dengan subset kolom terbatas sebagai kunci.

In [25]:
# Perform the first merge: merge_default
merge_default = pd.merge(sales_and_managers, revenue_and_sales)

# Print merge_default
merge_default

,city,state,units,branch,branch_id,manager,revenue
0,Mendocino,CA,1,Mendocino,47.0,Brett,200.0
1,Denver,CO,4,Denver,20.0,Joel,83.0
2,Austin,TX,2,Austin,10.0,Charlers,100.0


In [26]:
# Perform the second merge: merge_outer
merge_outer = pd.merge(sales_and_managers, revenue_and_sales, how='outer')

# Print merge_outer
merge_outer

,city,state,units,branch,branch_id,manager,revenue
0,Mendocino,CA,1,Mendocino,47.0,Brett,200.0
1,Denver,CO,4,Denver,20.0,Joel,83.0
2,Austin,TX,2,Austin,10.0,Charlers,100.0
3,Springfield,MO,5,Springfield,31.0,Sally,NaN
4,Springfield,IL,1,NaN,NaN,NaN,NaN
5,Springfield,IL,1,NaN,30.0,NaN,4.0
6,Springfield,MO,5,NaN,NaN,NaN,NaN


In [27]:
# Perform the third merge: merge_outer_on
merge_outer_on = pd.merge(sales_and_managers, revenue_and_sales, how='outer', on=['city', 'state'])

# Print merge_outer_on
merge_outer_on

,city,state,units_x,branch,branch_id_x,manager,branch_id_y,revenue,units_y
0,Mendocino,CA,1,Mendocino,47.0,Brett,47.0,200.0,1
1,Denver,CO,4,Denver,20.0,Joel,20.0,83.0,4
2,Austin,TX,2,Austin,10.0,Charlers,10.0,100.0,2
3,Springfield,MO,5,Springfield,31.0,Sally,NaN,NaN,5
4,Springfield,IL,1,NaN,NaN,NaN,30.0,4.0,1


**Note** : Perhatikan bagaimana `merge_default` menghilangkan baris `Springfield`, sedangkan `merge_outer` menyertakannya dua kali.

## Ordered merges

### Using merge_ordered()

Latihan ini menggunakan DataFrames `austin` dan `houston` yang masing-masing berisi data cuaca dari kota-kota Austin dan Houston. 

Kondisi cuaca direkam pada hari yang berbeda dan Anda harus menggabungkan kedua DataFrame ini bersama-sama sehingga tanggalnya diurutkan. Untuk melakukan ini, Anda akan menggunakan `pd.merge_ordered()`. Setelah selesai, perhatikan urutan baris sebelum dan sesudah penggabungan.

In [29]:
# Create DataFrames
austin = pd.DataFrame({ 'date' : ['2016-01-01', '2016-02-08', '2016-01-17'], 'ratings' : ['Cloudy', 'Cloudy', 'Sunny'] })
houston = pd.DataFrame({ 'date' : ['2016-01-04', '2016-01-01', '2016-03-01'], 'ratings' : ['Cloudy', 'Cloudy', 'Sunny'] })

# Print data
print(austin)
print(houston)

         date ratings
0  2016-01-01  Cloudy
1  2016-02-08  Cloudy
2  2016-01-17   Sunny
         date ratings
0  2016-01-04  Cloudy
1  2016-01-01  Cloudy
2  2016-03-01   Sunny


In [30]:
# Perform the first ordered merge: tx_weather
tx_weather = pd.merge_ordered(austin, houston)

# Print tx_weather
print(tx_weather)

# Perform the second ordered merge: tx_weather_suff
tx_weather_suff = pd.merge_ordered(austin, houston, on='date', suffixes=['_aus','_hus'])

# Print tx_weather_suff
print(tx_weather_suff)

# Perform the third ordered merge: tx_weather_ffill
tx_weather_ffill = pd.merge_ordered(austin, houston, on='date', suffixes=['_aus','_hus'], fill_method='ffill')

# Print tx_weather_ffill
print(tx_weather_ffill)

         date ratings
0  2016-01-01  Cloudy
1  2016-01-04  Cloudy
2  2016-01-17   Sunny
3  2016-02-08  Cloudy
4  2016-03-01   Sunny
         date ratings_aus ratings_hus
0  2016-01-01      Cloudy      Cloudy
1  2016-01-04         NaN      Cloudy
2  2016-01-17       Sunny         NaN
3  2016-02-08      Cloudy         NaN
4  2016-03-01         NaN       Sunny
         date ratings_aus ratings_hus
0  2016-01-01      Cloudy      Cloudy
1  2016-01-04      Cloudy      Cloudy
2  2016-01-17       Sunny      Cloudy
3  2016-02-08      Cloudy      Cloudy
4  2016-03-01      Cloudy       Sunny


**Note** : Perhatikan bagaimana setelah menggunakan metode `fill`, tidak ada lagi entri `NaN`.

### Using merge_asof()

Mirip dengan `pd.merge_ordered()`, fungsi `pd.merge_asof()` juga akan menggabungkan nilai secara berurutan menggunakan kolom `on`, tetapi untuk setiap baris di DataFrame kiri, hanya baris dari DataFrame kanan yang nilai kolom `'on'` nya kurang dari nilai kiri akan disimpan.

Fungsi ini dapat digunakan untuk menyelaraskan frekuensi datetime yang berbeda tanpa harus melakukan resample terlebih dahulu.

Di sini, Anda akan menggabungkan harga minyak bulanan (dolar AS) ke dalam dataset efisiensi bahan bakar mobil penuh. DataFrames oli dan mobil telah dimuat sebelumnya sebagai `oil` dan `auto`.

Kumpulan data ini akan menyelaraskan sedemikian rupa sehingga harga pertama tahun ini akan di broadcast ke dalam deretan DataFrame mobil. Ini dianggap benar karena pada awal tahun tertentu, sebagian besar mobil untuk tahun itu sudah dibuat.

Anda kemudian akan memeriksa DataFrame yang digabungkan, sampel ulang berdasarkan tahun dan menghitung rata-rata `'Price'` dan `'mpg'`. Anda harus dapat melihat tren dalam dua kolom ini, yang dapat Anda konfirmasikan dengan menghitung korelasi Pearson antara `'Price'` dan `'mpg'` yang diresampel.

In [60]:
# Assign url
url_oil = 'https://assets.datacamp.com/production/repositories/516/datasets/707566cf46c4dd6290b9029f5e07a92baf3fe3f7/oil_price.csv'
url_automobile = 'https://assets.datacamp.com/production/repositories/516/datasets/2f3d8b2156d5669fb7e12137f1c2e979c3c9ce0b/automobiles.csv'

# Load data
oil = pd.read_csv(url_oil, parse_dates=['Date'])
auto = pd.read_csv(url_automobile, parse_dates=['yr'])

In [61]:
auto.head()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name
0,18.0,8,307.0,130,3504,12.0,1970-01-01,US,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,1970-01-01,US,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,1970-01-01,US,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,1970-01-01,US,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,1970-01-01,US,ford torino


In [62]:
oil.head()

,Date,Price
0,1970-01-01,3.35
1,1970-02-01,3.35
2,1970-03-01,3.35
3,1970-04-01,3.35
4,1970-05-01,3.35


In [67]:
# Merge auto and oil: merged
merged = pd.merge_asof(auto, oil, left_on='yr', right_on='Date')

# Print the tail of merged
merged.tail()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name,Date,Price
387,27.0,4,140.0,86,2790,15.6,1982-01-01,US,ford mustang gl,1982-01-01,33.85
388,44.0,4,97.0,52,2130,24.6,1982-01-01,Europe,vw pickup,1982-01-01,33.85
389,32.0,4,135.0,84,2295,11.6,1982-01-01,US,dodge rampage,1982-01-01,33.85
390,28.0,4,120.0,79,2625,18.6,1982-01-01,US,ford ranger,1982-01-01,33.85
391,31.0,4,119.0,82,2720,19.4,1982-01-01,US,chevy s-10,1982-01-01,33.85


In [68]:
# Resample merged: yearly
yearly = merged.resample('A', on='Date')[['mpg','Price']].mean()

# Print yearly
yearly

,mpg,Price
Date,,
1970-12-31,17.689655,3.35
1971-12-31,21.111111,3.56
1972-12-31,18.714286,3.56
1973-12-31,17.100000,3.56
1974-12-31,22.769231,10.11
1975-12-31,20.266667,11.16
1976-12-31,21.573529,11.16
1977-12-31,23.375000,13.90
1978-12-31,24.061111,14.85


In [70]:
# print yearly.corr()
yearly.corr()

,mpg,Price
mpg,1.000000,0.948677
Price,0.948677,1.000000


**Note** : Sepertinya ada korelasi kuat antara mil per galon dan harga minyak!